# Рекомендация тарифов мобильной связи

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В нашем распоряжении данные о поведении клиентов, которые уже перешли на тарифы, которые мы исследовали в одном из прошлых проектов.

Теперь нам необходимо построить модель для задачи классификации, которая выберет подходящий новый тариф для пользователей, которые в настоящее время пользуются архивными тарифами.

Необходимо достигнуть точности больше 0.75.

## Изучение файла

Импортируем необходимые библиотеки:

In [253]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

Согласно инструкции, в нашем распоряжении датасет, который находится по пути "/datasets/users_behavior.csv". Импортируем его и выведем первые 10 строк.

In [254]:
df = pd.read_csv("/datasets/users_behavior.csv")
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0000,311.9000,83.0000,"19,915.4200",0
1,85.0000,516.7500,56.0000,"22,696.9600",0
2,77.0000,467.6600,86.0000,"21,060.4500",0
3,106.0000,745.5300,81.0000,"8,437.3900",1
4,66.0000,418.7400,1.0000,"14,502.7500",0
5,58.0000,344.5600,21.0000,"15,823.3700",0
6,57.0000,431.6400,20.0000,"3,738.9000",1
7,15.0000,132.4000,6.0000,"21,911.6000",0
8,7.0000,43.3900,3.0000,"2,538.6700",1
9,90.0000,665.4100,38.0000,"17,358.6100",0


Согласно инструкции, столбцы означают следующее:

- `сalls` — количество звонков.
- `minutes` — суммарная длительность звонков в минутах.
- `messages` — количество sms-сообщений.
- `mb_used` — израсходованный интернет-трафик в Мб.
- `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

**Вывод:**

В данном пункте был создан датасет с данными, которые представляют собой информацию о том, сколько минут, сообщений и интернет-трафика используют пользователи и каким тарифом они пользуются. С этими данными мы и будем работать в последующих пунктах. 

## Извлечение выборок

Далее необходимо разделить исходные данные на обучающую, валидационную и тестовую выборки. 

Согласно предоставленной на курсе теории, в таком случае данные делят в отношении 3:1:1. То есть, 60 % данных уходит на обучающую выборку, и по 20 % на валидационную и тестовую. Для этого воспользуемся функцией `train_test_split` из библитеки `sklearn`.

Сначала разделим датасет на обучающую выборку и валидационную с тестовой.

In [255]:
train, test_valid = train_test_split(df, train_size=0.6, random_state=123456789)

Теперь разобьем второй датасет на две выборки, валидационную и тестовую.

In [256]:
valid, test = train_test_split(test_valid, train_size=0.5, random_state=123456789)

Теперь разобьем все выборки на `features` и `target`:

- `features` будет представлять собой исходный датасет без целевого признака `is_ultra`.
- `target` - целевой признак `is_ultra`

Так как алгоритм в данном случае будет одинаков, предлагается написать для этого функцию.

In [257]:
def features_target(df):
    features = df.drop(['is_ultra'], axis=1)
    target = df['is_ultra']
    return features, target

Разделим все выборки, вызвав функцию для каждой из них.

In [258]:
train_features, train_target = features_target(train)
valid_features, valid_target = features_target(valid)
test_features, test_target = features_target(test)

**Вывод:**

В данном пункте мы разделили наши три датасета (обучающий, валидационный и тестовый) на два (с признаками и с целевым признаком). Далее можно переходить непосредственно к обучению.

## Исследование моделей

Теперь построим модель на наших данных. Для этого будем использовать три модели, а именно: 
- `DecisionTreeClassifier` или **дерево решений**.
- `RandomForestClassifier` или **случайный лес**. 
- `LogisticRegression` или **логистическая регрессия**. 

Для каждой модели мы будем подбирать оптимальные гиперпараметры путем оценки значения `accuracy`, то есть **точности**.

### Дерево решений

Оценим в цикле точность модели дерева решений для валидационной выборки при различных значениях гиперпараметра, а именно, величины `max_depth` или **глубины дерева**. 

Выведем на экран значения точности для разных значений глубины дерева.

In [259]:
for depth in range(1,6):
    model = DecisionTreeClassifier(random_state=123456789, max_depth=depth)
    model.fit(train_features, train_target)
    predictions = model.predict(valid_features)
    results = accuracy_score(valid_target, predictions)
    print(f'max_depth = {depth}, accuracy = {results}')

max_depth = 1, accuracy = 0.7589424572317263
max_depth = 2, accuracy = 0.807153965785381
max_depth = 3, accuracy = 0.8242612752721618
max_depth = 4, accuracy = 0.8180404354587869
max_depth = 5, accuracy = 0.8180404354587869


Видно, что оптимальным является глубина дерева 3.

### Случайный лес

В данном случае на валидационной выборке будем оптимизировать два параметра модели случайного леса, а именно `n_estimators` или **количества деревьев в лесу** и `max_depth` или **глубину леса**. 

Глубину предлагается перебирать в диапазоне от 1 до 5, а количество деревьев перебирать в диапазоне от 10 до 100 с шагом 10.

In [260]:
best_model = None
best_result = 0
for depth in range(1,6):
    for estimators in range(10, 101, 10):
        model = RandomForestClassifier(random_state=123456789, n_estimators=estimators, max_depth=depth)
        model.fit(train_features, train_target)
        predictions = model.predict(valid_features)
        results = accuracy_score(valid_target, predictions)
        if results > best_result:
            best_model = model
            best_result = results

Выведем значения точности и оптимизируемых параметров для наилучшей модели.

In [261]:
print(f'''Наилучшая точность в {best_result} в модели случайного леса достигается при глубине леса 
{best_model.max_depth} и количесте деревьев {best_model.n_estimators}''')

Наилучшая точность в 0.8289269051321928 в модели случайного леса достигается при глубине леса 
4 и количесте деревьев 40


### Логистическая регрессия

В данной модели нет оптимизируемых параметров. Предлагается просто посчитать ее точность на валидационной выборке.

In [262]:
model = LogisticRegression(random_state=123456789)
model.fit(train_features, train_target)
predictions = model.predict(valid_features)
result = accuracy_score(valid_target, predictions)
result

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7682737169517885

**Вывод:**

В данном пункте были найдены оптимальные параметры трех моделей на валидационной выборке и посчитана их точность:
- Модель `DecisionTreeClassifier` с параметром `max_depth` = 3 позоляет получить точность 0.824. 
- Модель `RandomForestClassifier` с параметром `max_depth` = 4 и `n_estimators` = 40 позоляет получить точность 0.829.
- В модели `LogisticRegression` оптимизация не проводилась. Рассчитанная точность модели составила 0.768.

## Проверка моделей на тестовой выборке

Проверим точность каждой из моделей с оптимальными параметрами на тестовой выборке.

Так как это повторяемый шаг, напишем для этого функцию.

In [263]:
def test_accuracy(model, train_features, train_target, test_features, test_target):
    model.fit(train_features, train_target)
    predictions = model.predict(test_features)
    results = accuracy_score(test_target, predictions)
    return results

Теперь посчитаем точность для каждой модели.

### Дерево решений

In [264]:
model = DecisionTreeClassifier(random_state=123456789, max_depth=3)
test_accuracy(model, train_features, train_target, test_features, test_target)

0.7931570762052877

### Случайный лес

In [265]:
model = RandomForestClassifier(random_state=123456789, n_estimators=40, max_depth=4)
test_accuracy(model, train_features, train_target, test_features, test_target)

0.80248833592535

### Логистическая регрессия

In [266]:
model = LogisticRegression(random_state=123456789)
test_accuracy(model, train_features, train_target, test_features, test_target)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7480559875583204

**Вывод:**

Полученные модели с оптимальными параметрами позволяют достигнуть следующую точность на тестовой выборке:
- 0.793 для `DecisionTreeClassifier`.
- 0.802 для `RandomForestClassifier`.
- 0.748 для `LogisticRegression`

Согласно условиям заказчика, необходимо было разработать модель с точностью больше 0.75. Соответственно, первые две модели подходят под данное требование. Максимальная точность была получена для модели `RandomForestClassifier`.

## Проверка моделей на адекватность

В данном пункте предлагается проверить модели на адекватность путем использования модели `DummyClassifier` из библиотеки `sklearn`.

Данная "фиктивная" модель предсказывает значения, не учитывая передаваемых ей признаков. В качестве стратегии предсказания выберем `most_frequent`, которая возвращает наиболее часто встречающееся значения.

In [267]:
model = DummyClassifier(strategy="most_frequent")
test_accuracy(model, train_features, train_target, test_features, test_target)

0.7013996889580093

**Вывод:**

Видно, что разработанная модель обладает точностью 0.802, а фиктивная модель - точностью 0.701. То есть, наша модель все же работает лучше фиктивной, значит ее можно принять.

## Общий вывод

В данном проекте была разработана модель, которая делает предсказания о том, на какой из новых тарифов лучше перейти абоненту, который в настоящее время использует архивный тариф.

1. Было проведено изучение данных, с которыми мы впоследствии работали.
2. Исходные данные были разделены на три выборки: обучающую, валидационную и тестовую в соотношении 3:1:1.
3. Были исследованы три модели, подобраны оптимальные параметры для некоторых из них и рассчитана точность моделей на валидационной выборке:
    - Для модели `DecisionTreeClassifier` с параметром `max_depth` = 3 точность составила 0.824.
    - Для модели `RandomForestClassifier` с параметром `max_depth` = 4 и `n_estimators` = 40 точность составила 0.829.
    - Для модели `LogisticRegression` точность составила 0.768.
4. Точность полученных моделей была проверены на тестовой выборке и составила:
    - 0.793 для модели `DecisionTreeClassifier`.
    - 0.802 для модели `RandomForestClassifier`.
    - 0.748 для модели `LogisticRegression`.
5. Полученные точности также были сравнены с простейшей моделью `DummyClassifier`, которая предсказыает значения не учитывая передаваемые ей признаки. Ее точность составила 0.701, что выше разработанных моделей.    


Согласно условиям заказчика, точность модели должна быть больше 0.75. Поэтому была принята модель с максимальной точностью, а именно, `RandomForestClassifier` с параметрами `max_depth` = 4 и `n_estimators` = 40.   
